## Homework 3
## Xin Ou (xo2119)  Xiaoyu Shi (xs2375)

## Setting for notebook and show the Python version

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import platform
platform.python_version()

'3.7.4'

## Import Scrapy

In [2]:
try:
    import scrapy
except:
    !pip install scrapy
    import scrapy
from scrapy.crawler import CrawlerProcess

## Setup a pipeline 

This class creates a simple pipeline that writes all found items to a JSON file, where each line contains one JSON element.

In [3]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('Legoresultt.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

## Define the spider

The LegosSpider class defines from which URLs to start crawling and which values to retrieve. I set the logging level of the crawler to warning, otherwise the notebook is overloaded with DEBUG messages about the retrieved data. I will extract the information includes 'Name','Pieces','Tags','Year' of each page.

In [4]:
import logging

class LegosSpider(scrapy.Spider):
    name = "lego"
    start_urls = ['https://brickset.com/sets/theme-Disney']
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'Legoresultt.json'                        # Used for pipeline 2
    }
    
    def parse(self, response):
        for LEGO in response.css('.set'):
            yield {
                'Name': LEGO.css('h1 ::text').extract_first(),
                'Pieces': LEGO.xpath('.//dl[dt/text() = "Pieces"]/dd/a/text()').extract_first(),
                'Tags': LEGO.css('div.tags span a::text').extract(),
                'Year': LEGO.css('div.tags a.year::text').extract()             
            }
            
        NEXT_PAGE_SELECTOR = '.next a ::attr(href)'
        next_page = response.css(NEXT_PAGE_SELECTOR).extract_first()
        if next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
                callback=self.parse
            )

## Start the crawler

In [5]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(LegosSpider)
process.start()

2020-02-17 13:52:18 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-02-17 13:52:18 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.4 (default, Aug 13 2019, 15:17:50) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.7, Platform Darwin-19.0.0-x86_64-i386-64bit
2020-02-17 13:52:18 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'Legoresultt.json', 'LOG_LEVEL': 30, 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}


## Check the files

Verify that the files has been created on disk. As we can observe the files are both created and have data. The .jl file has line separated JSON elements, while the .json file has one big JSON array containing all the legos.

In [6]:
ll Legoresultt.*

-rw-r--r--  1 kelanox  staff  14681 Feb 17 13:52 Legoresultt.jl
-rw-r--r--@ 1 kelanox  staff  14754 Feb 17 13:52 Legoresultt.json


In [7]:
!tail -n 2 Legoresultt.jl

{"Name": "Cogsworth", "Pieces": null, "Tags": ["Disney", " France", " In Store Build", " Toys R Us", " Brick Built Figure", " Beauty And The Beast"], "Year": ["2016", "2016"]}
{"Name": "Lumiere", "Pieces": null, "Tags": ["Disney", " France", " Lumiere", " In Store Build", " Toys R Us", " Brick Built Figure", " Beauty And The Beast"], "Year": ["2016", "2016"]}


In [8]:
!tail -n 2 Legoresultt.jl

{"Name": "Cogsworth", "Pieces": null, "Tags": ["Disney", " France", " In Store Build", " Toys R Us", " Brick Built Figure", " Beauty And The Beast"], "Year": ["2016", "2016"]}
{"Name": "Lumiere", "Pieces": null, "Tags": ["Disney", " France", " Lumiere", " In Store Build", " Toys R Us", " Brick Built Figure", " Beauty And The Beast"], "Year": ["2016", "2016"]}


## Create dataframes
Pandas can now be used to create dataframes and save the frames to pickles. The .json file can be loaded directly into a frame, whereas for the .jl file we need to specify the JSON objects are divided per line.

In [9]:
import pandas as pd
dfjson = pd.read_json('Legoresultt.json')
dfjson

,Name,Pieces,Tags,Year
0,Rapunzel's Market Visit,37.0,"[Rapunzel, Polybag, Disney, Cart, Baked Go...","[2014, 2014]"
1,Olaf's Summertime Fun,48.0,"[Olaf, Polybag, Disney, Musical, Beach, S...","[2016, 2016]"
2,Cinderella's Kitchen,37.0,"[Cinderella, Polybag, Disney, Kitchen, Cin...","[2017, 2017]"
3,Ariel's Underwater Symphony,51.0,"[Ariel, Polybag, Disney, Musical, Piano, ...","[2018, 2018]"
4,Elsa's Winter Throne,42.0,"[Elsa, Polybag, Disney, Ice, Frozen, Roya...","[2019, 2019]"
...,...,...,...,...
66,Aurora,NaN,[Aurora],"[2020, 2020]"
67,Arendelle Castle,NaN,"[Microscale, Castle, Disney]","[2019, 2019]"
68,Olaf Box,NaN,[],"[2019, 2019]"
69,Cogsworth,NaN,"[Disney, France, In Store Build, Toys R Us,...","[2016, 2016]"


## Explains which database you would use to store this data.

Answer: I will use the relational database to store this data. From the final dataframe, we can make some conclusions:
- A relational database is one that presents information in tables with rows and columns. 
- A table is a collection of objects of the same type(rows). 
- The ability to retrieve these related data provides us the term relational database.

I choose PostgreSQL (often referred to as Postgres) which is an object-relational database management system (ORDBMS) with an emphasis on extensibility and standards-compliance. PostgreSQL also has a number of JSON functions and operators that can be used with its two JSON data types (JSON, and JSONB).

PostgreSQL is also a relational database that is much more concerned with standards compliance and extensibility than with giving you freedom over how you store data. It uses both dynamic and static schemas and allows you to use it for relational data and normalized form storage. 